In [2]:
import os
from tqdm.auto import tqdm
import tensorflow as tf
from tensorflow import keras
import keras_cv
from keras_cv import bounding_box
#from keras_cv import visualizations
import numpy as np
import matplotlib.pyplot as plt

In [3]:
img_path = r'C:\Users\ue\OneDrive - IMS Messsysteme GmbH\Uni\MasterArbeit\Data\Batch1\Batch_1'
labels_path = r'C:\Users\ue\OneDrive - IMS Messsysteme GmbH\Uni\MasterArbeit\Data\Batch1\Batch_1_Labels_BBs'

In [ ]:
from tensorflow.keras.models import Sequential


def build_image_detection_pieline(img_size=(128,128), batch_size=32):
    classes = ['class_0','class_A','class_B','class_C','class_D','class_E','class_F']
    class_mapping = dict(zip(range(len(classes)), classes))
    
    def parse_labels(label_path):
        #read label file without tf
        label = open(label_path, 'r').read().split('\n')
        
        class_ids = []
        bbs = []
        for line in label:
            if line == '':
                continue
            #first string before space is the class, after that is the bounding box
            class_name = tf.strings.split(line, sep=' ')[0]
            class_id = classes.index(class_name)
            #read bounding box without tf
            bb = line.split(' ')[1:]
            bb = [float(i) for i in bb]
            #clip values to 0 and 1
            bb = [max(0,i) for i in bb]
            bb = [min(1,i) for i in bb]
            x1 = bb[0]*img_size[0]
            y1 = bb[1]*img_size[1]
            x2 = bb[2]*img_size[0]
            y2 = bb[3]*img_size[1]
            #print([x1,y1,x2,y2])
            class_ids.append(class_id)
            bbs.append([x1,y1,x2,y2])
            
            #class_name = tf.strings.to_number(class_name)
            
        return class_ids, bbs
    
    def parse_images(image_path):
        image = tf.io.read_file(image_path)
        image = tf.image.decode_png(image, channels=1) 
        image = tf.image.resize(image, img_size)
        return image
    
    def load_dataset(img_files,id,bb):
        image = parse_images(img_files)
        bounding_boxes = {
            "classes": tf.cast(id, dtype=tf.int64),
            "boxes": bb,
                        }
        return {"images": tf.cast(image, tf.int64), "bounding_boxes": bounding_boxes}
   
    
    def dict_to_tuple(inputs):
        return inputs["images"], inputs["bounding_boxes"]

    
    files = sorted(os.listdir(img_path))
    files = [os.path.join(img_path, file) for file in files]
    label_files = sorted(os.listdir(labels_path))
    label_files = [os.path.join(labels_path, label) for label in label_files]
    class_ids = []
    bbs = []
    for label in tqdm(label_files):
        class_id, bb = parse_labels(label)
        class_ids.append(class_id)
        bbs.append(bb)
    class_ids = tf.ragged.constant(class_ids)
    bbs = tf.ragged.constant(bbs)
    dataset = tf.data.Dataset.from_tensor_slices((files,class_ids,bbs))
    dataset = dataset.shuffle(len(files))

    train_size = int(0.7 * len(files))
    val_size = int(0.3 * len(files))
    train_ds = dataset.take(train_size)
    val_ds = dataset.skip(train_size)
    
    augmentor = keras.Sequential(
        layers=[
            #keras_cv.layers.Resizing(128,128, bounding_box_format="xyxy"),
            keras_cv.layers.RandomFlip(mode="horizontal", bounding_box_format="xyxy"),
            keras_cv.layers.RandomShear(x_factor=0.1, y_factor=0.1, bounding_box_format="xyxy"),
            keras_cv.layers.JitteredResize(
                target_size=(128,128), scale_factor=(0.75, 1.3), bounding_box_format="xyxy"
            ),
        ]
    )
    resizing = keras_cv.layers.JitteredResize(
        target_size=(128,128),
        scale_factor=(0.75, 1.3),
        bounding_box_format="xyxy",
    )
    
    train_ds = train_ds.map(load_dataset, num_parallel_calls=tf.data.AUTOTUNE)
    train_ds = train_ds.ragged_batch(batch_size, drop_remainder=True)
    train_ds = train_ds.map(augmentor, num_parallel_calls=tf.data.AUTOTUNE)
    train_ds = train_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
    train_ds = train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
    
    val_ds = val_ds.map(load_dataset, num_parallel_calls=tf.data.AUTOTUNE)
    val_ds = val_ds.ragged_batch(batch_size, drop_remainder=True)
    val_ds = val_ds.map(resizing, num_parallel_calls=tf.data.AUTOTUNE)
    val_ds = val_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
    val_ds = val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
    
    
    return train_ds, val_ds, class_mapping

train_ds, val_ds, class_mapping = build_image_detection_pieline()
    

2025-03-13 12:09:46.899494: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741864186.914315   27630 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741864186.918397   27630 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-13 12:09:46.932867: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


NameError: name 'os' is not defined

In [ ]:
import cv2
import matplotlib.pyplot as plt
#visualize images and bounding boxes in train_ds
inputs = next(iter(train_ds.take(1)))
image = inputs[0]
bounding_boxes = inputs[1]
bounding_boxes = bounding_boxes['boxes']
bounding_boxes = bounding_boxes.numpy()
bounding_boxes = bounding_boxes[0]

window = cv2.namedWindow('image', cv2.WINDOW_NORMAL)
cv2.resizeWindow('image', 600,600)
cv2.imshow('image', image[0])
#draw bounding boxes
for bb in bounding_boxes:
    bb = bb.numpy()
    bb = bb.astype(int)
    image = cv2.rectangle(image[0], (bb[0],bb[1]), (bb[2],bb[3]), (255,255,255), 2)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
classes = ['class_0','class_A','class_B','class_C','class_D','class_E','class_F']
class_mapping = dict(zip(range(len(classes)), classes))
print(class_mapping)
ids=list(class_mapping.keys())[list(class_mapping.values()).index('class_E')]
print(ids)


In [ ]:
classes = ['class_0','class_A','class_B','class_C','class_D','class_E','class_F']
id = classes.index('class_E')
print(id)